In [2]:
# load telegram export file
import json

with open('dialogs/result.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [6]:
len(data['messages'])

60400

In [23]:
# get random message
import random
random.choice(data['messages'])

{'id': 294141,
 'type': 'message',
 'date': '2022-09-27T23:20:06',
 'date_unixtime': '1664310006',
 'from': 'Сергей Холодильник',
 'from_id': 'user254968821',
 'text': 'Как ебнуться к 19 гадам краткий курс бля',
 'text_entities': [{'type': 'plain',
   'text': 'Как ебнуться к 19 гадам краткий курс бля'}]}

In [53]:
# get random chunk of consecutive messages
# random.sample(data['messages'], 4) # random sample (but not consecutive)

def get_random_chunk(data, chunk_size=4):
    start_index = random.randint(0, len(data['messages']) - chunk_size)
    return data['messages'][start_index:start_index+chunk_size]

get_random_chunk(data)

[{'id': 160410,
  'type': 'message',
  'date': '2022-03-08T03:26:30',
  'date_unixtime': '1646699190',
  'from': 'Сергей Холодильник',
  'from_id': 'user254968821',
  'text': 'Но у него они явно новые. Имхо у него к ним всё было. И новые моники, с тачем, без, клавы, радиаторы, куллеры, корпуса разные',
  'text_entities': [{'type': 'plain',
    'text': 'Но у него они явно новые. Имхо у него к ним всё было. И новые моники, с тачем, без, клавы, радиаторы, куллеры, корпуса разные'}]},
 {'id': 160413,
  'type': 'message',
  'date': '2022-03-08T03:27:48',
  'date_unixtime': '1646699268',
  'from': 'Cyber Potato',
  'from_id': 'user1049586649',
  'text': 'Мне нужен этот чел\nИ буду копить на малинку',
  'text_entities': [{'type': 'plain',
    'text': 'Мне нужен этот чел\nИ буду копить на малинку'}]},
 {'id': 160415,
  'type': 'message',
  'date': '2022-03-08T03:29:28',
  'date_unixtime': '1646699368',
  'from': 'Сергей Холодильник',
  'from_id': 'user254968821',
  'text': [{'type': 'link', 't

# Get good samples

In [27]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained('tinkoff-ai/response-quality-classifier-base')
model = AutoModelForSequenceClassification.from_pretrained('tinkoff-ai/response-quality-classifier-base')
inputs = tokenizer('[CLS]привет[SEP]привет![SEP]как дела?[RESPONSE_TOKEN]норм, у тя как?', max_length=128, add_special_tokens=False, return_tensors='pt')
with torch.inference_mode():
    logits = model(**inputs).logits
    probas = torch.sigmoid(logits)[0].cpu().detach().numpy()
relevance, specificity = probas

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [28]:
relevance, specificity

(0.6259156, 0.44406536)

In [54]:
def to_text(obj):
    # obj can be a string or a list of strings / dicts
    if isinstance(obj, str):
        return obj
    elif isinstance(obj, list):
        return ' '.join([to_text(o) for o in obj])
    elif isinstance(obj, dict):
        return to_text(obj['text'])
    else:
        return ''

In [100]:
def cleanup_chunk(chunk):
    chunk = [to_text(c) for c in chunk]
    chunk = [c for c in chunk if c != '' and len(c) < 512]
    return None if len(chunk) < 4 else chunk

In [76]:
def get_normal_chunk(data):
    sample = []
    while not sample:
        sample = cleanup_chunk(get_random_chunk(data))
    return sample


get_normal_chunk(data)


['Денис....',
 'Я дошёл до той стадии, когда у меня в голове реализация кода на си будет проще чем в питоне.....',
 'АХВХАХАХ СЕРЕГА',
 'Это клиника']

In [102]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# globals for classification
tokenizer = AutoTokenizer.from_pretrained(
    'tinkoff-ai/response-quality-classifier-base')
model = AutoModelForSequenceClassification.from_pretrained(
    'tinkoff-ai/response-quality-classifier-base')

# globals for toxicity
tokenizer_tox = AutoTokenizer.from_pretrained(
    'tinkoff-ai/response-toxicity-classifier-base')
model_tox = AutoModelForSequenceClassification.from_pretrained(
    'tinkoff-ai/response-toxicity-classifier-base')


In [98]:
def get_relevance_specificity(msg_list):
    if len(msg_list) != 4:
        return None
    inputs = tokenizer('[CLS]{}[SEP]{}[SEP]{}[RESPONSE_TOKEN]{}'.format(
        *msg_list), add_special_tokens=False, return_tensors='pt')
    with torch.inference_mode():
        logits = model(**inputs).logits
        probas = torch.sigmoid(logits)[0].cpu().detach().numpy()
    return probas


def cool_format(n, r, s, t):
    msgs = '\n'.join('- ' + m for m in n)
    print(f"{msgs}\n\nrelevance: {r*100:.2f}%, specificity: {s*100:.2f}%")


norm_chunk = get_normal_chunk(data)
relevance, specificity = get_relevance_specificity(norm_chunk)
cool_format(norm_chunk, relevance, specificity)


- Ты на локалке хостить будешь??
- Ну да?
- А на кой чёрт мне сервак ещё
- Кстати потом покажи как там это делается, с генерацией ssl сертификата

relevance: 71.63%, specificity: 80.51%


# Приколы

In [101]:
from tqdm import trange

aver_relevance, aver_specificity = 0, 0

for i in trange(1000):
    norm_chunk = get_normal_chunk(data)
    relevance, specificity = get_relevance_specificity(norm_chunk)
    aver_relevance += relevance
    aver_specificity += specificity

aver_relevance /= 1000
aver_specificity /= 1000

print(f"Average relevance: {aver_relevance*100:.2f}%, average specificity: {aver_specificity*100:.2f}%")

100%|██████████| 1000/1000 [01:44<00:00,  9.53it/s]

Average relevance: 34.47%, average specificity: 29.89%


In [111]:
def get_toxicity(msg_list):
    if len(msg_list) != 4:
        return None
    inputs = tokenizer_tox(
        '[CLS]{}[SEP]{}[SEP]{}[RESPONSE_TOKEN]{}'.format(*msg_list),
        add_special_tokens=False, return_tensors='pt')
    with torch.inference_mode():
        logits = model_tox(**inputs).logits
        probas = torch.sigmoid(logits)[0].cpu().detach().numpy()
    return probas


chunk = get_normal_chunk(data)
relevance, specificity = get_relevance_specificity(chunk)
toxicity = get_toxicity(chunk)


def cool_format(n, r, s, t):
    msgs = '\n'.join('- ' + m for m in n)
    print(f"{msgs}\n\nrelevance: {r*100:.2f}%, specificity: {s*100:.2f}%")
    # print("ok: {.2f}, toxic: {.2f}, risks: {.2f}, severe_toxic: {.2f}".format(*t))
    print(f"ok: {t[0]*100:.2f}%, toxic: {t[1]*100:.2f}%, risks: {t[2]*100:.2f}%, severe_toxic: {t[3]*100:.2f}%")


cool_format(chunk, relevance, specificity, toxicity)


- Блять, почему я при использовании веб фреймворка должен установить python2.7 и С компилятор
- А реально зачем?
- АХАХАХАХХААХАХАХАХАХАХАХАХАХАХАХХАХАХА
- ЛАДНО АХАХАХАХ

relevance: 24.37%, specificity: 13.96%
ok: 99.82%, toxic: 13.90%, risks: 8.88%, severe_toxic: 28.01%


# Приколы 2

In [122]:
aver_toxicity = [0, 0, 0, 0]

for i in trange(1000):
    chunk = get_normal_chunk(data)
    # relevance, specificity = get_relevance_specificity(chunk)
    toxicity = get_toxicity(chunk)
    relevance, specificity = get_relevance_specificity(chunk)
    # aver_toxicity = [x + y for x, y in zip(aver_toxicity, toxicity)]
    # aver_toxicity = [x + 1 for x, y in zip(aver_toxicity, toxicity) if y == max(toxicity)]
    for i in range(4):
        if toxicity[i] == max(toxicity):
            if i == 3:
                cool_format(chunk, relevance, specificity, toxicity)
                print()
            aver_toxicity[i] += 1

aver_toxicity = [x / 1000 for x in aver_toxicity]

print(f"Average toxicity: ok: {aver_toxicity[0]*100:.2f}%, toxic: {aver_toxicity[1]*100:.2f}%, risks: {aver_toxicity[2]*100:.2f}%, severe_toxic: {aver_toxicity[3]*100:.2f}%")

  2%|▏         | 18/1000 [00:03<03:10,  5.16it/s]

- Вот о чём я говорил кста
- Можешь распечатать один листок самостоялки Черняк
- Я попробую порешаю
- А то на телефоны агрится

relevance: 25.52%, specificity: 21.31%
ok: 68.48%, toxic: 4.76%, risks: 14.32%, severe_toxic: 97.88%



  3%|▎         | 27/1000 [00:05<03:38,  4.46it/s]

- О, реально годно
- И под мобилку оптимизировано
- и всё для докера😍
- Просто в одну команду ебануть и всё

relevance: 30.36%, specificity: 35.91%
ok: 82.59%, toxic: 1.41%, risks: 43.71%, severe_toxic: 93.73%



  3%|▎         | 33/1000 [00:07<03:57,  4.07it/s]

...

- ?????????
- Кого ты там обрезал
- ну я евнух теперь получается
- И да, экспорти их в jpg

relevance: 18.09%, specificity: 21.22%
ok: 14.28%, toxic: 21.62%, risks: 31.84%, severe_toxic: 94.21%



 98%|█████████▊| 975/1000 [03:45<00:06,  4.10it/s]

- Катя, Юля, Я, Ты
- Нормас
- Лера сказала что не смотрит такое((
- Кидай скам ссылку

relevance: 28.10%, specificity: 24.67%
ok: 6.28%, toxic: 10.22%, risks: 53.52%, severe_toxic: 97.24%



100%|██████████| 1000/1000 [03:51<00:00,  4.32it/s]

Average toxicity: ok: 94.10%, toxic: 0.30%, risks: 1.20%, severe_toxic: 4.40%


In [119]:
sum(aver_toxicity) * 100

100.0

In [123]:
aver_toxicity

[0.941, 0.003, 0.012, 0.044]

# Split into chunks

In [1]:
from telegram_models import TelegramExport

chats = TelegramExport.parse_file('result.json').chats

In [ ]:
from telegram_models import Message
from pydantic import BaseModel
from typing import List

class MessagesChunk(BaseModel):
    context: List[Message]
    summary: str
    response: Message
    relevance: float
    specificity: float
    toxicity: float

class DialogueChunk(BaseModel):
    messages: List[Message]

In [2]:
timings_list = []  # list of pairs: (message time, next message time)
my_name = 'Cyber Potato'

for chat in chats:
    for i in range(len(chat.messages) - 1):
        if chat.messages[i].from_ == my_name and chat.messages[i + 1].from_ != my_name:
            timings_list.append((chat.messages[i].date, chat.messages[i + 1].date))

In [3]:
# timings_list - array of pairs: (message time, next message time)
# graph of the altair dependency graph of the interval between the initial message and the next message on the time of day

import pandas as pd
from math import log2

timings = [
    {
        'time': t.hour,
        'delta': 2 ** round(log2((t2 - t).total_seconds() + 1)),
    }
    for t, t2 in timings_list
]

# count appearance of each time and delta
timings = pd.DataFrame(timings).groupby(['time', 'delta']).size().reset_index(name='count')

print(timings)


     time    delta  count
0       0        1    125
1       0        2    229
2       0        4    678
3       0        8   1248
4       0       16   1072
..    ...      ...    ...
498    23    65536     30
499    23   131072     11
500    23   262144      5
501    23   524288      3
502    23  1048576      5

[503 rows x 3 columns]


In [4]:
import altair as alt

alt.Chart(timings).mark_rect().encode(
    x='time:O',
    y=alt.Y('delta:O', scale=alt.Scale(type='log')),
    color=alt.Color('count:Q', scale=alt.Scale(type='log'))
)

alt.Chart(...)

In [50]:
chat = next(chat for chat in chats if chat.name == 'Сергей')
all_messages = pd.DataFrame([{
    'time': m.date.hour * 3600 + m.date.minute * 60 + m.date.second,
    'date': m.date.date(),
    'day': m.date.weekday(),
    'text': m.text,
    'from': m.from_,
} for m in chat.messages])

all_messages

,time,date,day,text,from
0,41286,2021-10-09,5,https://stackoverflow.com/questions/11334371/c...,Сергей Холодильник
1,41895,2021-10-09,5,https://drive.google.com/drive/folders/1Nb-14r...,Сергей Холодильник
2,45131,2021-10-09,5,https://stackoverflow.com/questions/238603/how...,Сергей Холодильник
3,54486,2021-10-09,5,int len = 0;\n\nchar *readstr()\n{\n len = 0;...,Cyber Potato
4,71612,2021-10-09,5,Сохрани это как компромат,Cyber Potato
...,...,...,...,...,...
45927,2860,2022-12-05,0,что блять,Cyber Potato
45928,2862,2022-12-05,0,помянем,Cyber Potato
45929,2864,2022-12-05,0,"Превью хорошее, реалистичное",Сергей Холодильник
45930,2930,2022-12-05,0,Прикол у меня есть русский и я сделала трениро...,Сергей Холодильник


In [51]:
# get slice of 1 week
from datetime import date, timedelta

week = all_messages[(all_messages.date >= date(2022, 11, 1)) & (all_messages.date < date(2022, 11, 8))]
week

,time,date,day,text,from
36050,62082,2022-11-01,1,Жить жизнь,Cyber Potato
36051,62082,2022-11-01,1,лаба по физике,Cyber Potato
36052,62082,2022-11-01,1,,Cyber Potato
36053,62082,2022-11-01,1,,Cyber Potato
36054,62082,2022-11-01,1,,Cyber Potato
...,...,...,...,...,...
37641,84862,2022-11-07,0,Да,Сергей Холодильник
37642,84866,2022-11-07,0,ахуеть,Cyber Potato
37643,84872,2022-11-07,0,Очень давно,Сергей Холодильник
37644,84876,2022-11-07,0,Еще с яндекс радио,Сергей Холодильник
